<a href="https://colab.research.google.com/github/grizzay-dev/Burst_Analysis/blob/master/tf_df_burst_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install pandas

In [1]:
import tensorflow_decision_forests as tfdf
import numpy as np
import pandas
import time

ModuleNotFoundError: ignored

In [ ]:
train_df = pandas.read_csv("ttree_output3.csv")

def split(ds, r=0.3):
    test_ind = np.random.rand(len(ds)) < r
    return ds[~test_ind], ds[test_ind]

#split data
np.random.seed(12)
burst_train, burst_test = split(train_df)

In [ ]:
print(burst_train)
print(burst_test)

In [ ]:
#Convert the pandas dataframe into a TensorFlow dataset
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(burst_train, label="burst_type")

# Train the model
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

In [ ]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(burst_test, label="burst_type")

model.compile(metrics=["accuracy"])
print(model.evaluate(test_ds))

model.save("models/burst_model")

In [ ]:
#Print model summary
model.summary()

In [ ]:
#Plot tree
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=1, max_depth=3)

In [ ]:
tree = model.make_inspector().extract_tree(0)
tfdf.model_plotter.plot_tree(tree)
print(tree.pretty())

with open('readme.txt', 'w') as f:
    f.write(tree.pretty())

In [ ]:
#var importance
model.make_inspector().variable_importances()

In [ ]:
#plot training
import matplotlib.pyplot as plt

logs = model.make_inspector().training_logs()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1, 2, 2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.show()